# Import

In [ ]:
# Standard libraries
import os
import sys

# Data processing
import numpy as np
import pandas as pd
from PIL import Image

# Deep learning
import torch

# Visualization
import matplotlib
from IPython.display import display

# Project-specific imports
sys.path.insert(0, './atarihead')
sys.path.insert(0, '../src')
import data_reader as ahdr
from replay_buffer import HDF5ReplayBufferRAM as HDF5ReplayBuffer, Transition

# Configuration
pd.options.display.float_format = '{:.3g}'.format

# IPython display setup
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display


# Load CSV

In [ ]:
game_name='...'

folder_path = fr"atari_agents\records\MDQN_modern\{game_name}\0"

# Load other info
csv_path=folder_path+r'\reward_action_table.csv'
data = pd.read_csv(csv_path)
data.tail()

data['new_frames'] = data['frame_until'].diff().fillna(data['frame_until'].iloc[0])

if game_name=='NameThisGame': # NameThisGame exhibits some unrecorded but counted frames at the beginning of each episode
    data[data['new_frames']>4]=4

data['frame_number']=data['frame_until']-data['frame_until'].values[0]
data.head()

C:\Users\henri\AppData\Local\Temp\ipykernel_22004\360332904.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  data[data['new_frames']>4]=4


# Load images

In [ ]:
folder_path = fr"atari_agents\records\MDQN_modern\{game_name}\0"
print(folder_path)


images = []
image_names = []

idf=0
for filename in sorted(os.listdir(folder_path)):
    if idf%5000==0:
        print(idf)
    if filename.endswith('.png'):
        # Full path to the image
        image_path = os.path.join(folder_path, filename)
        
        if idf in data['frame_number'].values:

            img = Image.open(image_path)
            img_array = np.array(img, dtype=np.uint8)[:,::2,:]
            
            images.append(img_array)
            image_names.append(filename)

        idf+=1

# Now images contains numpy arrays and image_names contains the corresponding names


atari_agents\records\MDQN_modern\NameThisGame\0
0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000
195000
200000
205000
210000
215000
220000
225000
230000
235000
240000
245000
250000
255000
260000
265000
270000
275000
280000
285000
290000
295000
300000
305000
310000
315000
320000
325000
330000
335000
340000
345000
350000
355000
360000
365000
370000
375000
380000
385000
390000
395000
400000


In [ ]:
# Create state, action, reward tensors
states = images

action_vals_np=data['action'].values
action_vals = torch.tensor(action_vals_np.astype(np.int32), dtype=torch.int32).unsqueeze(1)

reward_vals_np=data['reward'].values
reward_vals = torch.tensor(reward_vals_np.astype(np.float32), dtype=torch.float32).unsqueeze(1)

done_vals=data['done'].values

In [ ]:
# Create replay buffer
memory = HDF5ReplayBuffer(
    file_path='atari_agents/replay_buffers/'+game_name+'_atari_agents_buffer_100k_4f.h5',
    initial_capacity=100000,
    state_shape=(210,160,3),
    action_shape=(1,),
    gaze_shape=(2,),
)

for i in range(len(action_vals)):
    if i%10000==0:
        print(i)
    state = states[i]

    next_state = states[i+1] if i+1 < len(states) else states[i]

    memory.push(state=state,action=action_vals[i],next_state=next_state,reward=reward_vals[i],frame_id=image_names[i],gaze_pos=None,done=done_vals[i])

memory.close()

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000


IndexError: list index out of range